# Recurrent neural network (RNN) from scratch

## What is a recurrent neural network?

A recurrent neural network (RNN), as opposed to a regular fully connected neural network (FCNN), has layers that are connected to themselves.

The difference might be clearer by first looking at an FCNN.

<img src="Figures/fcnn.svg" width="40%" alt="FCNN">

In an FCNN there are no connections between nodes in a single layer. For instance, $h_1^1$ is not connected to $h_2^1$. In addition, the input and output are always of a fixed length.

In an RNN, however, this is no longer the case. Nodes in the hidden layers are connected to themselves, represented by the curved lines in the figure below.

<img src="Figures/rnn.svg" width="40%">

Thus the output $\vec{h}$ from the hidden layer is fed back into the hidden layer. This recurrence makes RNNs useful when working with sequential data, as we can have input of variable length. This is more clear if we unfold the recurrent part of the network.

<img src="Figures/rnn_unfold.svg" width="40%">

## The mathematics of RNNs

### Feed forward

Consider some sequential input $X$ with $n$ features. Note that $X$ here is an array with two axes, since it contains $n$ features at each time step in the sequence. We will denote the input at a specific time step $t$ as
$$\vec{X}^{(t)} = \begin{pmatrix}
X^{(t)}_1 \\ \vdots \\ X^{(t)}_n
\end{pmatrix},$$
which is then an $n$-dimensional vector.

Next, consider an RNN with $L$ hidden layers, and an output layer with $m$ features. We will denote the output of the $l$'th hidden layer at time step $t$ as
$$\vec{h}_l^{(t)} = \begin{pmatrix}
h_{l, 1}^{(t)} \\ \vdots \\ h_{l, n_l}^{(t)}
\end{pmatrix},$$
with $n_l$ being the number of features in the $l$'th hidden layer. The output of the RNN at time step $t$ is denoted
$$\hat{\vec{y}}^{(t)} = \begin{pmatrix}
\hat{y}_1 \\ \vdots \\ \hat{y}_m,
\end{pmatrix}$$
where the hat is there to distinguish the RNN output $\hat{\vec{y}}^{(t)}$ from the target value, which is denoted $\vec{y}^{(t)}$.
The RNN will then look like this.

<img src="Figures/large_rnn.svg" width="50%">

<img src="Figures/feed_forward.svg" width="90%">